# Librerias

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, precision_recall_curve,recall_score, f1_score,  make_scorer
from scipy.stats import ks_2samp, kruskal, chisquare
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import cufflinks as cf
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from sklearn.manifold import MDS
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier
from varclushi import VarClusHi
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.feature_selection import SelectKBest, VarianceThreshold
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import pandas as pd

import plotly.express as px 
import numpy as np

import autopep8
import os
import warnings
warnings.filterwarnings("ignore")


cf.go_offline()
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Carga de datos

In [ ]:
path = '/home/david/AMV/Ciencia_Datos/Proyecto_Final_Salary/'

In [ ]:
os.listdir(path)

In [ ]:
df = pd.read_csv(path + 'salary.csv')

# No Supervisado 

## Analisis Exploratorio

### Ausentes

In [ ]:
df.isna().sum() 

**Sin ausentes**

### Duplicados

In [ ]:
df.shape, df.dropna().shape 

**Sin duplicados**

## Ingenieria de datos

In [ ]:
df.shape 

In [ ]:
df.head(3)

In [ ]:
df.columns

### Variables Continuas

In [ ]:
df = df.rename(columns ={'age':'x_age',
                    'fnlwgt':'x_fnlwgt',
                    'education-num':'x_education-num',
                    'capital-gain':'x_capital-gain',
                    'capital-loss':'x_capital-loss',
                    'hours-per-week':'x_hours-per-week'})

In [ ]:
df['x_years_exp'] = (df['x_age'] - df['x_education-num']) /df['x_age']

In [ ]:
l = []
for c in range(len(df)):
    if df['x_capital-gain'][c] >0 or df['x_capital-loss'][c] >0:
        l.append(1)
    else:
        l.append(0)

In [ ]:
df['x_invest'] = l 

In [ ]:
varc = list(df.filter(like = 'x_').columns )

In [ ]:
varc 

In [ ]:
df[varc] 

In [ ]:
df[varc].describe()

### Variables Discretas

In [ ]:
vard = [v for v in list(df.columns) if v not in varc] 

**Estas nos van a servir para el perfilamiento de los clusters una vez acabado, cuando se le dio un enfoque de aprendizaje supervisado, estas variables fueron tratadas para convertirlas a variables continuas con el uso de la variable objetivo para sacar el porcentaje de ocurrencia de objetivo segun cada valor de la variable discreta y reemplazarlo (Se partio el espacio en Train y validate antes para este proceso), pero en este caso no sera necesario y la variable objetivo pasa a formar parte de las variables discretas**

### Extremos

In [ ]:
df[varc].hist(figsize=(10,10)) 

In [ ]:
ext = df[varc].describe(percentiles=[0.01,0.99]).T[['1%','99%']].reset_index()

for v, li, ls in ext.values:
    df[f'ol_{v}'] = ( (df[v]<li) | (df[v]>ls) ).astype(int)

df['ext'] = df.filter(like='ol').max(axis=1)
df.drop(df.filter(like='ol_').columns, axis=1, inplace=True )

df['ext'].value_counts(True)

In [ ]:
df = df.loc[ df['ext'] == 0 ].drop(['ext'],axis=1)

In [ ]:
df[varc].hist(figsize=(10,10)) 

## Analisis Bivariado

In [ ]:
df[varc].corr()

## Multicolinealidad

In [ ]:
vc = VarClusHi(df=df,feat_list=varc).varclus().rsquare.sort_values( by=['Cluster',
                                                                        'RS_Ratio'] , 
                                                                   ascending=[1,1] ).reset_index(drop=True)



vc['id'] = vc.groupby('Cluster').cumcount()+1 


vc 

**Nuestras variables redujeron sus numeros de 8 a 6 descartando 2 que nos ofrecian la misma información**

In [ ]:
best = vc.loc[vc['id'] == 1]['Variable'].tolist() 
best  

## Reducción de Dimensiones

In [ ]:
X = df[best] 

In [ ]:
sc=StandardScaler()
X_sc = sc.fit_transform(X)

In [ ]:
mm=MinMaxScaler()
X_mm = mm.fit_transform(X) 

### AF

In [ ]:
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity

In [ ]:
chi2,p = calculate_bartlett_sphericity(X_sc)
print("Esfericidad de Bartlett")
print("Valor de Chi : ",chi2)
print("P - value : ",p) 

In [ ]:
from factor_analyzer.factor_analyzer import calculate_kmo 

In [ ]:
kmo_all,kmo_model = calculate_kmo(X_sc)
print("KMO Test Statisitc",kmo_model)  

**KMO menor a .6**

### PCA

In [ ]:
from sklearn.decomposition import PCA 

In [ ]:
pca = PCA(n_components=3)
pca.fit(X_sc) 

In [ ]:
X_pca = pd.DataFrame(pca.fit_transform(X_sc))

In [ ]:
X_pca.columns=["componente_1","componente_2","componente_3"] 

In [ ]:
fig = px.scatter_3d(X_pca, x='componente_1', y='componente_2', z='componente_3')
fig.show() 

### Kernel PCA

In [ ]:
X.shape

In [ ]:
from sklearn.decomposition import  KernelPCA 

In [ ]:
sc=StandardScaler()
X_sc1= sc.fit_transform(X[:5_000])
X_sc2= sc.fit_transform(X[5_000:10_000])
X_sc3= sc.fit_transform(X[10_000:15_000])
X_sc4= sc.fit_transform(X[15_000:20_000])
X_sc5= sc.fit_transform(X[20_000:25_000])
X_sc6= sc.fit_transform(X[25_000:])

In [ ]:
kernel_pca = KernelPCA(n_components=3, kernel="rbf",gamma=1e-3,alpha=5e-3,fit_inverse_transform=True) 

In [ ]:
X_kpca = pd.DataFrame()
for x in [X_sc1, X_sc2, X_sc3,X_sc4, X_sc5, X_sc6]:
    X_kpca = pd.concat( [X_kpca, pd.DataFrame(kernel_pca.fit_transform(x)) ],ignore_index=True ) 

In [ ]:
X_kpca.columns=["componente_1","componente_2","componente_3"] 

In [ ]:
fig = px.scatter_3d(X_kpca, x='componente_1', y='componente_2', z='componente_3')
fig.show() 

### TSNE

In [ ]:
from sklearn.manifold import TSNE  

In [ ]:
tsne = TSNE(n_components=3, random_state=0) 

In [ ]:
X_tsne = pd.DataFrame()
for x in [X_sc1, X_sc2, X_sc3,X_sc4, X_sc5, X_sc6]:
    X_tsne = pd.concat( [X_tsne, pd.DataFrame(tsne.fit_transform(x)) ],ignore_index=True ) 

In [ ]:
X_tsne.columns=["componente_1","componente_2",'componente_3']

In [ ]:
fig = px.scatter_3d(X_tsne, x='componente_1', y='componente_2', z='componente_3' )
fig.show()

## Número optimo de Clusters

### Codo

In [ ]:
from yellowbrick.cluster import KElbowVisualizer
model = KMeans()
visualizer = KElbowVisualizer(model, k=(1,12)).fit(X_sc)
visualizer.show()  

In [ ]:
inercia = []
for k in list(range(2, 10)):
    km = KMeans(n_clusters=k)
    km.fit(X_sc)
    inercia.append(km.inertia_)

In [ ]:
df_ine=pd.DataFrame()
df_ine["N_Clusters"]=range(2,10)
df_ine["Inercia"]=inercia 

In [ ]:
fig = px.line(df_ine, x="N_Clusters", y="Inercia", title="Número de clusters óptimo") 
fig.show() 

### Silueta

In [ ]:
from yellowbrick.cluster import SilhouetteVisualizer

In [ ]:
for i in range(2,10):
    km = KMeans(i, random_state=200)
    visualizer = SilhouetteVisualizer(km, colors='yellowbrick')
    visualizer.fit(X_sc)
    print(f"N Clusters : {i}")
    print(f"Score Silueta : {round(visualizer.silhouette_score_,2)}")
    visualizer.show()  

In [ ]:
silueta = []
for k in list(range(2, 10)):
    km = KMeans(n_clusters=k)
    km.fit(X_sc)
    score = silhouette_score(X_sc, km.labels_, metric='euclidean')
    silueta.append(score)

In [ ]:
df_silueta=pd.DataFrame()
df_silueta["N_Clusters"]=range(2,10)
df_silueta["score"]=silueta 

In [ ]:
fig = px.line(df_silueta, x="N_Clusters", y="score", title="Número de clusters óptimo - Silueta") 
fig.show()  

### Calinski Harabasz

In [ ]:
from sklearn.metrics import calinski_harabasz_score

In [ ]:
calinski = []
for k in list(range(2, 10)):
    km = KMeans(n_clusters=k)
    km.fit(X_sc)
    labels = km.fit_predict(X_sc)
    score = calinski_harabasz_score(X_sc,labels)
    calinski.append(score) 

In [ ]:
df_calinski=pd.DataFrame()
df_calinski["N_Clusters"]=range(2,10)
df_calinski["score"]=calinski

In [ ]:
fig = px.line(df_calinski, x="N_Clusters", y="score", title="Número de clusters óptimo - Calinski") 
fig.show()

### Daves Bouldin

In [ ]:
from sklearn.metrics import davies_bouldin_score 

In [ ]:
bouldin = []
for k in list(range(2, 10)):
    km = KMeans(n_clusters=k)
    km.fit(X_sc)
    labels = km.fit_predict(X_sc)
    score = davies_bouldin_score(X_sc,labels)
    bouldin.append(score)

In [ ]:
df_bouldin=pd.DataFrame()
df_bouldin["N_Clusters"]=range(2,10)
df_bouldin["score"]=bouldin

In [ ]:
fig = px.line(df_bouldin, x="N_Clusters", y="score", title="Número de clusters óptimo - Davies  Bouldin") 
fig.show() 

## Clusters

### K Means

In [ ]:
from sklearn.cluster import KMeans  

In [ ]:
km = KMeans(n_clusters=5,init='random', random_state=300)

In [ ]:
km.fit(X_sc) 

In [ ]:
df_aux=df.copy()
df_aux["cl"]=km.predict(X_sc)   

In [ ]:
df_aux 

In [ ]:
df_aux['cl'].value_counts().plot(kind='pie') 

In [ ]:
X_pca['cl'] = X_tsne['cl'] =  df_aux['cl'].astype(str)

In [ ]:
fig = px.scatter_3d(X_pca, x='componente_1', y='componente_2', z='componente_3',
              color='cl')
fig.show() 

In [ ]:
fig = px.scatter_3d(X_tsne, x='componente_1', y='componente_2', z='componente_3',
              color='cl')
fig.show() 

### GM

In [ ]:
from sklearn.mixture import GaussianMixture 

In [ ]:
GMM=GaussianMixture(n_components=6,random_state=0)
GMM 

In [ ]:
df_aux=df.copy()
df_aux["cl"]= GMM.fit_predict(X_sc)  

In [ ]:
X_pca['cl'] = df_aux['cl'].astype(str) 

In [ ]:
df_aux['cl'].value_counts().plot(kind='pie')  

In [ ]:
fig = px.scatter_3d(X_pca, x='componente_1', y='componente_2', z='componente_3',
              color='cl')
fig.show() 

### DBSCAN

In [ ]:
from sklearn.neighbors import NearestNeighbors
neigh = NearestNeighbors(n_neighbors=3)
nbrs = neigh.fit(X_sc)
distancias,_ = nbrs.kneighbors(X_sc)
distancias = np.sort(distancias, axis=0) 
distancias = distancias[:,1] 

In [ ]:
aux=pd.DataFrame()
aux["distancias"]=sorted(list(distancias),reverse=False)
aux["index"]=range(len(distancias))   

In [ ]:
fig = px.line(aux, x="index", y="distancias", title="Valor óptimo de Epsilon") 
fig.show()  

In [ ]:
0.8 

In [ ]:
from sklearn.cluster import DBSCAN
dbscan_opt=DBSCAN(eps=0.6,min_samples=6)

In [ ]:
df_aux=df.copy()  
df_aux["cl"]= dbscan_opt.fit_predict(X_sc) 

In [ ]:
X_pca['cl'] = df_aux['cl'].astype(str) 

In [ ]:
fig = px.scatter_3d(X_pca, x='componente_1', y='componente_2', z='componente_3',
              color='cl')
fig.show()  

## Perfilamiento del modelo

In [ ]:
df['cl'] = km.fit_predict(X_sc)   

In [ ]:
df

In [ ]:
print('Medina Poblacional')
display(df[best].median().to_frame().T)  

In [ ]:
print('Media Poblacional')
display(df[best].mean().to_frame().T)  

In [ ]:
print('Media por Cluster')
display(df[['cl']+best].groupby('cl').mean())  

In [ ]:
vard_prob_dicts['education']

In [ ]:
vard_prob_dicts['relationship']